In [2]:
%run runaway_functionsv3
%matplotlib qt
# cl = Cluster('LP_1487')
# # cld = ClusterDias('Berkeley_97')
# # plot = cl.plot_cmd(isochrones=[Isochrone(cld)])
# plot = cl.plot_traceback_psr()
# show_annotation_manager(plot)


In [8]:
dias2021 = Table.read('dias2021.tsv', format='ascii.ecsv')
dias2021['logage'].min(), dias2021['logage'].max()

(6.668, 9.999)

In [2]:
%run runaway_functionsv3
%matplotlib qt
cl = Cluster('Alessi_Teutsch_5')
cl.plot_cmd()


<AxesSubplot: xlabel='$G_{BP}-G_{RP}$ (mag)', ylabel='$G$ (mag)'>

In [4]:
latex_text(cl)

NameError: name 'latex_text' is not defined

In [3]:
cl.plot_pm()
cl.plot_traceback_clean()
cl.plot_traceback_psr()

10pc image downloaded in 4.706837s
50pc image downloaded in 28.883331s


<WCSAxesSubplot: >

In [5]:
def latex_text(cluster, n_members=None):
    cld = ClusterDias(cluster.name)
    ra, dec = cluster.skycoord.ra, cluster.skycoord.dec
    ra_str, dec_str = ra.to_string(format='latex')[1:-1], dec.to_string(format='latex')[1:-1]
    dist_str = str(round(cluster.distance.value,1)) + r"\pm" + str(round(cluster.e_distance.value,1))
    
    intro_text = rf"""\subsection{{Introduction}}
    {cluster.name.replace('_', ' ')} is located at $\alpha = {ra_str}, \delta = {dec_str}$ at a distance of ${dist_str}$ pc.
    {cluster.N} out of the {cld.N} mentioned in Dias have been selected as members. Using these, the kinematic parameters of the cluster have been determined. These are compared to the kinematic parameters of the runaway star in \ref{{tab:{cluster.name}_kinematics}}. Some of the brightest memebrs fo the cluster are shown in table \ref{{tab:{cluster.name}_members}}.
    
    \subsection{{CMD and proper motion}}
    The parameters of the cluster's isochrone ...
    Using these parameters, the CMD plot for the cluster is shown in figure \ref{{fig:{cluster.name}_cmd_pm}} in the left. The right plot in the figure shows the proper motion diagram showing that the runaway moves with a proper motion that is different from the cluster.
    
    \begin{{figure}}
    \centering
    \includegraphics[width=0.49\linewidth]{{Results/{cluster.name}/{cluster.name}_cmd.pdf}}
    \includegraphics[width=0.49\linewidth]{{Results/{cluster.name}/{cluster.name}_pm.pdf}}
    \caption{{\\ 
    \textit{{Left}}: The CMD showing the temperature estimate of the runaway. The isochrone for the temperature estimate is shown in blue \\
    \textit{{Right}}: Proper motion diagram depicting that the star is an outlier in proper motion in the local rest frame.}}
        \label{{fig:{cluster.name}_cmd_pm}}
    \end{{figure}}    

    \subsection{{Traceback}}
    
    The runaway star's motion relative to the cluster is shown in figure \ref{{fig:{cluster.name}_traceback.pdf}}.
    
    \begin{{figure}}
    \centering
    \includegraphics[width=0.49\linewidth]{{Results/{cluster.name}/{cluster.name}_traceback_clean.pdf}}
    \includegraphics[width=0.49\linewidth]{{Results/{cluster.name}/{cluster.name}_pm.pdf}}
    \caption{{\\ 
    \textit{{Left}}: The runaway star ...'s motion with respect to the cluster. The four green lines depict the four extreme cases of the proper motion considering the errors and the green ellipse represents the possible positions of the star 100 kyr ago. \\
    \textit{{Right}}: Proper motion diagram depicting that the star is an outlier in proper motion in the local rest frame.}}
        \label{{fig:{cluster.name}_traceback_clean&pm}}
    \end{{figure}}
    
    \subsection{{Traceback}}
    """
    
    # Members table generation
    table = cluster.mymembers
    table.sort('Gmag')
    table = table[:n_members]
    
    formatted_distances = []
    formatted_parallaxes = []
    formatted_gmags = []
    formatted_bprps = []
    
    for row in table:
        distance = row['rgeo']
        plx = row['Plx']
        e_plx = row['e_Plx']
        upper_error = row['B_rgeo'] - row['rgeo']
        lower_error = -row['b_rgeo'] + row['rgeo']
        gmag = row['Gmag']
        e_gmag = row['e_Gmag']
        bprp = row['BP-RP']
        e_bprp = row['e_BP-RP']
        
        formatted_distance = f"${distance:.0f}^{{+{upper_error:.0f}}}_{{-{lower_error:.0f}}}$"
        formatted_parallax = f"${plx:.4f}\pm{e_plx:.4f}$"
        formatted_gmag = f"${gmag:.3f}\pm{e_gmag:.3f}$"
        formatted_bprp = f"${bprp:.3f}\pm{e_bprp:.3f}$"
        
        formatted_distances.append(formatted_distance)
        formatted_parallaxes.append(formatted_parallax)
        formatted_gmags.append(formatted_gmag)
        formatted_bprps.append(formatted_bprp)

    table['formatted_distance'] = formatted_distances
    table['formatted_parallax'] = formatted_parallaxes
    table['formatted_gmag'] = formatted_gmags
    table['formatted_bprp'] = formatted_bprps
    
    t_dict_members = {
        'Gaia DR3 Source': table['Source'],
        r"$r_{\text{geo}}$ (pc)": table['formatted_distance'],
        r"$\pi$ (mas)": table['formatted_parallax'],
        r"$G$ (mag)": table['formatted_gmag'],
        r"$G_{\text{BP}}-G_{\text{RP}}$ (mag)": table['formatted_bprp']
    }
    
    latexdict_members = {
        'tabletype': 'table*',
        'tablealign': 'h',
        'header_start': r'\hline',
        'data_start': r'\hline',
        'data_end': r'\hline', 
        'caption': f'Selected members of {cluster.name.replace("_"," ")}',
        'preamble': r'\label{tab:' + f'{cluster.name}_members' + '}'
    }
    
    astropy_table_members = Table(t_dict_members)
    members_output_filename = 'members_text.txt'
    
    astropy.io.ascii.write(astropy_table_members, format='latex', output=members_output_filename, overwrite=True, latexdict=latexdict_members)
    
    with open(members_output_filename, 'r+') as f:
        lines = f.readlines()
        lines[1], lines[2] = lines[2], lines[1]
        f.seek(0)
        f.writelines(lines)
        members_table_text = ''.join(lines)
    
    os.remove(members_output_filename)
    kinematics_table_text = generate_kinematics_latex(cluster.name)
    # Combine intro_text, kinematics_table_text, and members_table_text
    full_text = intro_text + "\n\n" + kinematics_table_text + "\n\n" + members_table_text
    
    filename = f"{cluster.name}.tex"
    
    with open(filename, 'w') as file:
        file.write(full_text)
    
    return full_text

# Example usage
cl = Cluster('Alessi_Teutsch_5')
print(latex_text(cl, n_members=10))


NameError: name 'generate_kinematics_latex' is not defined

In [33]:
from astropy.table import Table, Column

# Example data
cluster_part = Table(cl.all)

# Create the combined columns
distance_column = [f"${dist}\pm{e_dist}$" for dist, e_dist in zip(cluster_part['Dist'], cluster_part['e_Dist'])]
pmRA_column = [f"${pmRA}\pm{e_pmRA}$" for pmRA, e_pmRA in zip(cluster_part['pmRA'], cluster_part['e_pmRA'])]
pmDE_column = [f"${pmDE}\pm{e_pmDE}$" for pmDE, e_pmDE in zip(cluster_part['pmDE'], cluster_part['e_pmDE'])]
# Add the new column to the table
cluster_part['d'] = Column(distance_column, unit=u.pc)
cluster_part['d'] = Column(distance_column, unit=u.pc)
cluster_part['d'] = Column(distance_column, unit=u.pc)
# cluster_part.remove_columns(['Dist','e_Dist'])
cluster_part.rename_column('RA_ICRS',r'$\alpha$')
cluster_part.rename_column('DE_ICRS',r'$\delta$')

# Display the table
# print(table)
cluster_part


Cluster,$\alpha$,$\delta$,r50,N,pmRA,e_pmRA,pmDE,e_pmDE,Plx,e_Plx,RV,e_RV,NRV,logage,e_logage,__Fe_H_,e__Fe_H_,Av,e_Av,FileName,SimbadName,_RA.icrs,_DE.icrs,d
,deg,deg,deg,,mas / yr,mas / yr,mas / yr,mas / yr,mas,mas,km / s,km / s,,log(yr),log(yr),,,mag,mag,,,deg,deg,pc
str17,float64,float64,float32,int16,float32,float32,float32,float32,float32,float32,float64,float32,int16,float32,float32,float32,float32,float32,float32,str31,str31,float64,float64,str9
Alessi_Teutsch_5,332.2744,61.0962,0.337,59,-2.052,0.379,-3.306,0.379,1.125,0.043,-23.489,11.137,39,7.071,0.054,-0.053,0.096,1.253,0.076,clusters1/Alessi_Teutsch_5.dat,Name Alessi-Teutsch 5,332.2744,61.0962,$31\pm31$


In [23]:
from astropy.io import ascii
import io

output = io.StringIO()
ascii.write(table, output, format='latex')
latex_string = output.getvalue()
output.close()

latex_string = latex_string.replace('RA_ICRS', r'$\alpha$')

print(latex_string)

\begin{table}
\begin{tabular}{ccccccccccccccccccccccccccc}
Cluster & $\alpha$ & DE_ICRS & r50 & N & pmRA & e_pmRA & pmDE & e_pmDE & Plx & e_Plx & RV & e_RV & NRV & Dist & e_Dist & logage & e_logage & __Fe_H_ & e__Fe_H_ & Av & e_Av & FileName & SimbadName & _RA.icrs & _DE.icrs & d \\
 & $\mathrm{{}^{\circ}}$ & $\mathrm{{}^{\circ}}$ & $\mathrm{{}^{\circ}}$ &  & $\mathrm{mas\,yr^{-1}}$ & $\mathrm{mas\,yr^{-1}}$ & $\mathrm{mas\,yr^{-1}}$ & $\mathrm{mas\,yr^{-1}}$ & $\mathrm{mas}$ & $\mathrm{mas}$ & $\mathrm{km\,s^{-1}}$ & $\mathrm{km\,s^{-1}}$ &  & $\mathrm{pc}$ & $\mathrm{pc}$ & log(yr) & log(yr) & $\mathrm{}$ & $\mathrm{}$ & $\mathrm{mag}$ & $\mathrm{mag}$ &  &  & $\mathrm{{}^{\circ}}$ & $\mathrm{{}^{\circ}}$ &  \\
Alessi_Teutsch_5 & 332.2744 & 61.0962 & 0.337 & 59 & -2.052 & 0.379 & -3.306 & 0.379 & 1.125 & 0.043 & -23.489 & 11.137 & 39 & 31 & 31 & 7.071 & 0.054 & -0.053 & 0.096 & 1.253 & 0.076 & clusters1/Alessi_Teutsch_5.dat & Name Alessi-Teutsch 5 & 332.2744 & 61.0962 & $31\pm31$ \\


In [ ]:
def generate_kinematics_latex(clustername):
    cl = Cluster(clustername)
    kinematic_table  = cl.all
    
    